In [1]:
%matplotlib notebook
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import xml.etree.ElementTree as ET
from IPython.display import Image
import random

In [6]:
# trip distribution by numbers
# merged_data.groupby(by=['COUNTY_NM', 'dest_TAZ'])['demand'].agg('sum')

In [7]:
# trip distribution by percentages
# merged_data.groupby(by=['COUNTY_NM', 'dest_TAZ'])['demand'].agg('sum')/merged_data.demand.sum() *100

COUNTY_NM  dest_TAZ 
King       King         50.854349
           Kitsap        0.046566
           Pierce        1.924806
           Snohomish     2.719326
Kitsap     King          0.044790
           Kitsap        5.885178
           Pierce        0.269283
           Snohomish     0.004914
Pierce     King          1.920831
           Kitsap        0.269929
           Pierce       17.268226
           Snohomish     0.010494
Snohomish  King          2.715734
           Kitsap        0.005005
           Pierce        0.010817
           Snohomish    16.049753
Name: demand, dtype: float64

1- kitsap-kitsap/pierce trips are not relevant to king county, and percent of trips between king/snohomish and kitsap is 0.046566+ 0.044790+0.004914+0.005005 = 0.01 % 
- therefore we fully eliminate trips to/from kitsap 

2- snohomish-pierce and pierce-snohomish that pass trough king county are negligible (0.010817+0.010494 = 0.02%)
- we eliminate trips between Snohomish and Pierce county

3- trips within pierce and snohomish are not relevant to king county
- we eliminate trips within Snohomish and Pierce county

### therefore, we only keep trips 1- within king county, 2- between king and pierce, and 3- between king and Snohomish



## For Pierce and Snohomish, 
it seems that majority of trips take place either between pierce and south of downtown OR snohomish and north of downtown, so what we do is keeping only the trips between snohomish/pierce and sumo TAZs

### final stats before further filterations



In [101]:
# filtered_data.groupby(by=['COUNTY_NM', 'dest_TAZ'])['demand'].agg('sum')

COUNTY_NM  dest_TAZ 
King       King         5040054
           Pierce        190763
           Snohomish     269506
Pierce     King            7352
Snohomish  King           18304
Name: demand, dtype: int64

In [10]:
# filtered_data.groupby(by=['COUNTY_NM', 'dest_TAZ'])['demand'].agg('sum')/filtered_data.demand.sum() *100

COUNTY_NM  dest_TAZ 
King       King         84.566906
           Pierce        3.200806
           Snohomish     4.522033
Pierce     King          3.194195
Snohomish  King          4.516059
Name: demand, dtype: float64

### Selecting TAZs and working with them in this notebook:
we run the code below in qgis python plugin:

<pre>
dir = '/Users/stlp/Library/CloudStorage/GoogleDrive-soheil99@uw.edu/My Drive/reasearch codes/SUMO/MatSumo-main/SUMO/demand generation/regions'
os.chdir(dir)
def get_selected_TAZs(address):
    layer = QgsProject.instance().mapLayersByName("taz2010")[0]
    selected_features = layer.selectedFeatures()
    attribute_list = [feature['TAZ'] for feature in selected_features]
    with open(address, 'w') as f:
        f.write(str(attribute_list))
        f.close()
</pre>
we select some regions in taz2010.shp and then use the command above like: `get_selected_TAZ('/1.txt')`
and read selected regions in qgis here using the function below:

In [2]:
def _read_region(address):
    with open(address, 'r') as f:
        content = f.read()
        data_list = eval(content)
    return data_list

def read_regions(address):
    if type(address)==list:
        outputs = []
        for adrs in address:
            outputs.append(_read_region(adrs))
        return outputs
    else:
        return _read_region(address)
            
#     with open(address, 'r') as f:
#         content = f.read()
#         data_list = eval(content)
#     return data_list

# Conversion algorithm
So far, I was trying to eliminate useless trips. There are some trips that we ARE going to use: trips with sumo_TAZs in one end (without over-3700 TAZs or kitsap TAZs)

I think the best way to deal with these trips is by starting with trips involving sumo_TAZs and then gradually add different scenarios in which vehicles will pass through sumo_TAZs.


In [3]:
# Load shapefiles
taz_data = gpd.read_file('modified shp file/taz2010.shp')

# Load CSV file
trip_data = pd.read_csv("psrc_vehicle_trips.csv")

# load sumo tazs
tree = ET.parse('correct_Alaskan_modified_new_Taz_with_pseudo.add.xml')
root = tree.getroot()
sumo_taz = []
for taz in root.findall('taz'):
    tazid = taz.get('id')
    if tazid != 'taz_13':
        sumo_taz.append(int(tazid))

In [4]:
# we obtain list of TAZs for each of counties so that we can use them during filtering single trips dataframe
king = list(taz_data[taz_data['COUNTY_NM']=='King'].TAZ.values)
pierce = list(taz_data[taz_data['COUNTY_NM']=='Pierce'].TAZ.values)
kitsap = list(taz_data[taz_data['COUNTY_NM']=='Kitsap'].TAZ.values)
snohomish = list(taz_data[taz_data['COUNTY_NM']=='Snohomish'].TAZ.values)

In [5]:
len(trip_data)

9985425

Eliminate over-3700s, Kitsap, and port trips

In [6]:
trip_data = trip_data[~(trip_data.otaz.isin(kitsap)|trip_data.dtaz.isin(kitsap))]
trip_data = trip_data[~((trip_data.otaz>3700)|(trip_data.dtaz>3700))]
ports = [444, 445, 449, 478]
trip_data = trip_data[~(trip_data.otaz.isin(ports)|trip_data.dtaz.isin(ports))]
len(trip_data)

9250192

## TAZ conversion for sumo_TAZs included trips

In [7]:
trip_data['original_o'] = trip_data.otaz
trip_data['original_d'] = trip_data.dtaz
trip_data['original_deptm'] = trip_data.deptm
# adding more columns will increase computation time in next steps

In [8]:
converted_data = trip_data[trip_data.otaz.isin(sumo_taz) | trip_data.dtaz.isin(sumo_taz)]
len(converted_data)

608908

In [9]:
duplicate_rows = trip_data[trip_data.duplicated(subset=['otaz', 'dtaz', 'deptm'], keep=False)]
print(duplicate_rows)

         otaz  dtaz  deptm  travtime  original_o  original_d  original_deptm
126         1     5    482         5           1           5             482
127         1     5    482         6           1           5             482
1244        2     1   1107         2           2           1            1107
1245        2     1   1107         5           2           1            1107
1290        2     2   1018         4           2           2            1018
...       ...   ...    ...       ...         ...         ...             ...
9331718  3466  3466   1013         3        3466        3466            1013
9331724  3466  3466   1040        22        3466        3466            1040
9331725  3466  3466   1040         1        3466        3466            1040
9331739  3466  3466   1123        52        3466        3466            1123
9331740  3466  3466   1123        16        3466        3466            1123

[567030 rows x 7 columns]


### for Pierce and Snohomish trips
Based on map of highways, we can assign pierce to I5-south (5002, 5003) and snohomish to I5-north (5000, 5001).  It is not exact and there could be many many trips passing through bellevue instead of I-5.

In [10]:
def random_split_dataframe(df, percentage):
    if percentage < 0 or percentage > 100:
        raise ValueError("Percentage should be between 0 and 100")

    # Shuffle the DataFrame to randomize the order
    df = df.sample(frac=1, random_state=1)

    # Calculate the number of rows for the first part (X%)
    num_rows = len(df)
    split_index = int(num_rows * percentage / 100)

    # Split the DataFrame into two parts
    part1 = df.iloc[:split_index, :]
    part2 = df.iloc[split_index:, :]

    return part1, part2

In [11]:
# interval = 15
def get_intra_TAZ_traveltime(o_list, d_list, interval=15, exact_time = None):
    if exact_time is None:
        trips = trip_data[trip_data.otaz.isin(o_list) & trip_data.dtaz.isin(d_list)]
        average = round(trips.travtime.mean())
        time_offsets = list()
        for start in range(0,1440, interval):
            end = start+interval
            interval_trips = trips[trips.deptm.isin(range(start, end))]
            average_time = interval_trips.travtime.mean()
            if pd.isna(average_time):
                time_offsets.append(average)
            else:
                time_offsets.append(round(average_time))
        return time_offsets
    else:
        trips = trip_data[trip_data.otaz.isin(o_list) & 
                          trip_data.dtaz.isin(d_list) &
                          trip_data.deptm.isin(range(int(exact_time-interval/2), int(exact_time+interval/2)+1))]

        average = trips.travtime.mean()
        return average

In [12]:
def get_random_TAZ(taz_list):
    if type(taz_list)==list:
        return random.choice(taz_list)
    else:
        return taz_list

In [13]:
def convert1(invalid_TAZs, near_pseudo_TAZs, pseudo_taz_in, pseudo_taz_out, df=None):
    if df is None:
            df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    invalid_to_near_tt = get_intra_TAZ_traveltime(invalid_TAZs, near_pseudo_TAZs)
#     print(f"it'll take {len(df)} iterations.")
    for index, row in tqdm(df.iterrows()):
        interval = int(row.deptm//15)
        if row.otaz in invalid_TAZs:
            row.otaz = get_random_TAZ(pseudo_taz_in)
            row.deptm += invalid_to_near_tt[interval]
        elif row.dtaz in invalid_TAZs:
            row.dtaz = get_random_TAZ(pseudo_taz_out)
        converted_data.loc[index, :] = row 

In [14]:
sumo_s = read_regions('regions/sumo_south.txt')
sumo_n = [taz for taz in sumo_taz if taz not in sumo_s]
sumo_e = read_regions('regions/sumo_east.txt') 
sumo_w = [taz for taz in sumo_taz if taz not in sumo_e]
near_i90 = read_regions('regions/near_i90.txt')

In [15]:
Image(url="regions/i5.png", width=300, height=300)

In [16]:
near_i5n=[381, 382, 406, 407, 428, 430]
near_i5s=[649, 650, 657, 669, 670] 
nearSR99n = read_regions('regions/near_SR99N.txt')
near6001 = read_regions('regions/near6001.txt')

#v.3
southboundary = read_regions('regions/south_boundary.txt')
north1boundary = read_regions('regions/north1_boundary.txt')
north2boundary = read_regions('regions/north2_boundary.txt')
north3boundary = read_regions('regions/north3_boundary.txt')
east1boundary = read_regions('regions/east1_boundary.txt')
east2boundary = read_regions('regions/east2_boundary.txt')
industrial=read_regions('regions/version2/industrial.txt')


In [17]:
convert1(pierce, near_i5s, 5002, 5003)

14553it [01:50, 131.62it/s]


In [18]:
# convert1(snohomish, near_i5n, 5000, 5001)
# it'll take 36268 iterations

## newer stuff
--------

In [19]:
def north_trips_handler(invalid_TAZs):
    df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    print(f'total trips: {len(df)}')
    # if trips go to north1
#     north1=[426, 427, 428, 429]
    north1 = [426, 427, 428, 429, 549, 551, 553] #v.3
    df1 = df[df.otaz.isin(north1)|df.dtaz.isin(north1)]
    convert1(invalid_TAZs, near_i5n, [5000, 7001], [5001, 7001], df=df1)
#     convert1(invalid_TAZs, near_i5n, [5000, 7001], 5001,  df=df1) #v.3

    # if trips go to north2
    north21=[430, 432, 434]
    df1 = df[df.otaz.isin(north21)|df.dtaz.isin(north21)]
    convert1(invalid_TAZs, near_i5n, 7002, 7002, df=df1)

    north22 = [431, 433, 435]
    df1 = df[df.otaz.isin(north22)|df.dtaz.isin(north22)]
    convert1(invalid_TAZs, near_i5n, [5000, 7002], 7002, df=df1)


    # if trips go to north3
    north3=[436, 437, 438, 439, 440, 441, 442, 443]
    df1 = df[df.otaz.isin(north3)|df.dtaz.isin(north3)]
    convert1(invalid_TAZs, nearSR99n, [5010, 7003], [5011, 7003], df=df1)

    # if trips go to downtown north west
    df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    df1 = df[df.otaz.isin(sumo_w)|df.dtaz.isin(sumo_w)]
    df1 = df1[df1.otaz.isin(sumo_n)|df1.dtaz.isin(sumo_n)]
    
    df25, df75 = random_split_dataframe(df1, 25)
    convert1(invalid_TAZs, nearSR99n, 5010, 5011, df=df25)
    convert1(invalid_TAZs, near_i5n, 5000, 5001, df=df75)
    

    # other trips 
    convert1(invalid_TAZs, near_i5n, 5000, 5001)

In [20]:
north_trips_handler(snohomish)

total trips: 36268


1135it [00:08, 133.72it/s]
3634it [00:27, 134.36it/s]
2694it [00:19, 135.63it/s]
2127it [00:15, 134.93it/s]
2969it [00:22, 129.83it/s]
8910it [01:35, 93.76it/s] 
14799it [02:00, 122.55it/s]


In [21]:
def northeast_trips_handler(invalid_TAZs):
    df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    print(f'total trips: {len(df)}')
    # if trips go to north1
#     north1=[426, 427, 428, 429]
    north1 = [426, 427, 428, 429, 549, 551, 553] #v.3
    df1 = df[df.otaz.isin(north1)|df.dtaz.isin(north1)]
    convert1(invalid_TAZs, near_i5n, [5000, 7001], 5001, df=df1)

    # if trips go to north2
    north21=[430, 432, 434]
    df1 = df[df.otaz.isin(north21)|df.dtaz.isin(north21)]
    convert1(invalid_TAZs, near_i5n, 7002, 7002, df=df1)

    north22 = [431, 433, 435]
    df1 = df[df.otaz.isin(north22)|df.dtaz.isin(north22)]
    convert1(invalid_TAZs, near_i5n, [5000, 7002], 7002, df=df1)


    # if trips go to north3
    north3=[436, 437, 438, 439, 440, 441, 442, 443]
    df1 = df[df.otaz.isin(north3)|df.dtaz.isin(north3)]
    convert1(invalid_TAZs, nearSR99n, 7003, 7003, df=df1)

    # other trips 
    convert1(invalid_TAZs, near_i5n, 5000, 5001)

In [22]:
def northwest_trips_handler(invalid_TAZs):
    df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    print(f'total trips: {len(df)}')
    # if trips go to north1
#     north1=[426, 427, 428, 429]
    north1 = [426, 427, 428, 429, 549, 551, 553] #v.3
    df1 = df[df.otaz.isin(north1)|df.dtaz.isin(north1)]
    convert1(invalid_TAZs, near_i5n+nearSR99n, [5010, 7001], [5001, 7001], df=df1)

    # if trips go to north21
    north21=[430, 432, 434]
    df1 = df[df.otaz.isin(north21)|df.dtaz.isin(north21)]
#     convert1(invalid_TAZs, nearSR99n, 5010, [5011, 7002], df=df1)
    convert1(invalid_TAZs, nearSR99n, [5010, 7002], 7002, df=df1) #v.3
    
    # v.3 - if trips go to north22
    north22=[431, 433, 435]
    df1 = df[df.otaz.isin(north22)|df.dtaz.isin(north22)]
    convert1(invalid_TAZs, nearSR99n, [5010, 7002], 5011, df=df1)

    # if trips go to north32
    north3=[440, 441, 442, 443,446, 447, 448, 450, 451, 452, 453, 457, 458, 459, 460, 467, 468, 469]
    df1 = df[df.otaz.isin(north3)|df.dtaz.isin(north3)]
#     convert1(invalid_TAZs, nearSR99n+near6001, [5010, 6001], [5011, 6001], df=df1)
    #v.3
    df30, df70 = random_split_dataframe(df1, 30)
    convert1(invalid_TAZs, nearSR99n, 5010, 5011, df30)
    convert1(invalid_TAZs, near6001, 6001, 6001, df70)


    # if trips go to downtown east
    df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    df1 = df[df.otaz.isin(sumo_e)|df.dtaz.isin(sumo_e)]
    df20, df80 = random_split_dataframe(df1, 20)
    convert1(invalid_TAZs, near_i5n, 5000, 5001, df=df80)
    convert1(invalid_TAZs, nearSR99n, 5010, 5011, df=df20)

    # other trips 
#     convert1(invalid_TAZs, nearSR99n, 5010, 5011)
    df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    df = df[df.otaz.isin(sumo_n)|df.dtaz.isin(sumo_n)] #v.3 northwest trips
    df20, df80 = random_split_dataframe(df, 20)
    convert1(invalid_TAZs, nearSR99n, 5010, 5011, df80)
    convert1(invalid_TAZs, near6001, 6001, 6001, df20)
    
    df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    df20, df80 = random_split_dataframe(df, 20)
    convert1(invalid_TAZs, near_i5n, 5000, 5001, df=df20)
    convert1(invalid_TAZs, nearSR99n, 5010, 5011, df=df80)
    
    # could add one for industrial zone

### for trips within king county

In [23]:
Image(url="regions/version2/north1.png", width=300, height=300)

In [24]:
north_trips_handler(read_regions('regions/version2/north1.txt'))

total trips: 11749


424it [00:04, 95.74it/s] 
1131it [00:10, 105.59it/s]
922it [00:09, 96.93it/s] 
745it [00:06, 118.36it/s]
1019it [00:07, 130.04it/s]
3060it [00:23, 131.91it/s]
4448it [00:33, 131.42it/s]


In [25]:
Image(url="regions/version2/north2.png", width=300, height=300)

In [26]:
north_trips_handler(read_regions('regions/version2/north2.txt'))

total trips: 3878


123it [00:01, 101.35it/s]
453it [00:03, 117.59it/s]
363it [00:02, 136.57it/s]
183it [00:01, 137.04it/s]
342it [00:02, 137.04it/s]
1028it [00:07, 134.11it/s]
1386it [00:13, 106.27it/s]


In [27]:
Image(url="regions/version2/north3.png", width=300, height=300)

In [28]:
north_trips_handler(read_regions('regions/version2/north3.txt'))

total trips: 15273


537it [00:05, 97.54it/s] 
1499it [00:11, 127.71it/s]
1269it [00:09, 136.14it/s]
913it [00:07, 117.73it/s]
1324it [00:16, 82.07it/s] 
3974it [00:43, 92.40it/s] 
5757it [00:55, 103.45it/s]


In [29]:
Image(url="regions/version2/north4.png", width=300, height=300)

In [30]:
north_trips_handler(read_regions('regions/version2/north4.txt'))

total trips: 11794


461it [00:05, 92.08it/s]
1367it [00:14, 91.20it/s]
1097it [00:12, 85.03it/s] 
634it [00:05, 121.74it/s]
1011it [00:08, 123.37it/s]
3034it [00:25, 121.03it/s]
4190it [00:34, 122.87it/s]


In [31]:
Image(url="regions/version2/north5.png", width=300, height=300)

In [32]:
northeast_trips_handler(read_regions('regions/version2/north5.txt'))

total trips: 12334


460it [00:04, 111.96it/s]
1515it [00:12, 118.48it/s]
1227it [00:09, 126.32it/s]
631it [00:05, 124.58it/s]
8501it [01:10, 120.62it/s]


In [33]:
Image(url="regions/version2/north6.png", width=300, height=300)

In [34]:
north_trips_handler(read_regions('regions/version2/north6.txt'))

total trips: 21517


1425it [00:11, 124.42it/s]
2650it [00:21, 123.90it/s]
2097it [00:17, 118.28it/s]
1173it [00:09, 126.63it/s]
1732it [00:13, 129.05it/s]
5199it [00:49, 106.06it/s]
7241it [00:58, 123.35it/s]


In [35]:
Image(url="regions/version2/north7.png", width=300, height=300)

In [36]:
northwest_trips_handler(read_regions('regions/version2/north7.txt')+[337, 338, 356])

total trips: 19859


650it [00:05, 125.32it/s]
2875it [00:24, 116.37it/s]
2215it [00:17, 127.60it/s]
551it [00:04, 122.56it/s]
1288it [00:10, 121.40it/s]
1879it [00:15, 123.83it/s]
469it [00:03, 122.02it/s]
5341it [00:45, 118.50it/s]
1335it [00:11, 120.03it/s]
651it [00:06, 96.69it/s]
2605it [00:29, 88.06it/s]


In [37]:
Image(url="regions/version2/north8.png", width=300, height=300)

In [38]:
northwest_trips_handler(read_regions('regions/version2/north8.txt'))

total trips: 9358


208it [00:02, 82.65it/s]
1074it [00:11, 90.27it/s]
890it [00:09, 91.21it/s]
295it [00:03, 91.65it/s]
690it [00:07, 88.09it/s]
948it [00:10, 86.41it/s]
237it [00:02, 89.17it/s]
2450it [00:32, 74.95it/s]
612it [00:06, 100.90it/s]
390it [00:03, 101.17it/s]
1564it [00:16, 94.12it/s] 


In [39]:
Image(url="regions/version2/north9.png", width=300, height=300)

In [40]:
invalid_TAZs = read_regions('regions/version2/north9.txt')
df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
print(f'total trips: {len(df)}')

# if trips go to north3
north32=[436, 437, 438, 439, 440, 441, 442, 443]
df1 = df[df.otaz.isin(north32)|df.dtaz.isin(north32)]
df20, df80 = random_split_dataframe(df1, 20)
convert1(invalid_TAZs, near6001, 6001, 6001, df=df80)
convert1(invalid_TAZs, north3boundary, 7003, 7003, df=df20)

# if trips go to downtown south east
df1 = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
df1 = df1[df1.otaz.isin(sumo_e)|df1.dtaz.isin(sumo_e)]
convert1(invalid_TAZs, near_i5n, 5000, 5001, df=df1)

# other trips 
df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
df20, df80 = random_split_dataframe(df, 20)
convert1(invalid_TAZs, nearSR99n, 5010, 5011, df20)
convert1(invalid_TAZs, near6001, 6001,  6001, df80)

total trips: 12236


960it [00:09, 104.85it/s]
240it [00:02, 103.48it/s]
787it [00:07, 108.74it/s]
2049it [00:21, 97.35it/s] 
8200it [01:22, 99.03it/s] 


### V.3 new super TAZs

In [41]:
def uppereast_trips_handler(invalid_TAZs):
    
    df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    df= df[df.otaz.isin(sumo_e)|df.dtaz.isin(sumo_e)]
    # for sumo-northeast
    north1 = [426, 427, 428, 429, 547, 549, 551, 553] 
    df1 = df[df.otaz.isin(north1)|df.dtaz.isin(north1)]
    convert1(invalid_TAZs, near_i5n, [5000, 7001], [5001, 7001], df=df1)
    # for sumo-southeast
    df2 = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
    df20, df80 = random_split_dataframe(df2, 20)
    convert1(invalid_TAZs, near_i90, 5004, 5005, df=df80)
    convert1(invalid_TAZs, east2boundary, 7005, 7005, df=df20)
    # for rest of east
    df20, df80 = random_split_dataframe(df, 20)
    convert1(invalid_TAZs, near_i5n, 5000, 5001, df=df80)
    convert1(invalid_TAZs, east1boundary, 7004, 7004, df=df20)
    
    
    df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    df1 = df[df.otaz.isin(sumo_n)|df.dtaz.isin(sumo_n)]
    # for sumo-northwest
    convert1(invalid_TAZs, near_i5n, 5000, 5001, df=df1)
    
    # for sumo southwest
    df2 = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
    convert1(invalid_TAZs, near_i90, 5004, 5005, df=df2)

In [42]:
def east_trips_handler(invalid_TAZs):
    
    df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    df = df[df.otaz.isin(sumo_e)|df.dtaz.isin(sumo_e)]
    df = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
    df30, df70 = random_split_dataframe(df, 30)
    convert1(invalid_TAZs, east2boundary, 7005, 7005, df=df30)
    
    convert1(invalid_TAZs, near_i90, 5004, 5005)

In [43]:
def southeast_trips_handler(invalid_TAZs):
    df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    df = df[df.otaz.isin(sumo_e)|df.dtaz.isin(sumo_e)]
    df = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
    # assume: 40% of trips that use i90 will use 7005
    df1, df2 = random_split_dataframe(df, 50)
    df40, df60 = random_split_dataframe(df1, 40) 
    convert1(invalid_TAZs, east2boundary, 7005, 7005, df=df40)

    
    df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    df = df[df.otaz.isin(industrial)|df.dtaz.isin(industrial)]
    # assume: 40% of trips that use i5 will use 7000
    df1, df2 = random_split_dataframe(df, 50)
    df40, df60 = random_split_dataframe(df1, 40) 
    convert1(invalid_TAZs, southboundary, 7000,  7000, df=df40)
    
    df = converted_data[converted_data.otaz.isin(invalid_TAZs)|converted_data.dtaz.isin(invalid_TAZs)]
    df1, df2 = random_split_dataframe(df, 50)
    convert1(invalid_TAZs, near_i5s, 5002, 5003, df=df1)
    convert1(invalid_TAZs, near_i90, 5004, 5005, df=df2)

In [44]:
Image(url="regions/version3/farEast1.png", width=500, height=300)

In [45]:
uppereast_trips_handler(read_regions('regions/version3/farEast1.txt'))

12it [00:00, 87.57it/s]
59it [00:00, 77.91it/s]
14it [00:00, 51.08it/s]
94it [00:01, 82.80it/s]
23it [00:00, 87.46it/s]
324it [00:03, 100.94it/s]
148it [00:01, 118.37it/s]


In [46]:
Image(url="regions/version3/farEast2.png", width=400, height=300)

In [47]:
east_trips_handler(read_regions('regions/version3/farEast2.txt'))

131it [00:01, 92.65it/s]
3702it [00:30, 119.75it/s]


In [48]:
Image(url="regions/version3/east6.png", width=500, height=300)

In [49]:
uppereast_trips_handler(read_regions('regions/version3/east6.txt'))

168it [00:01, 112.82it/s]
356it [00:02, 125.52it/s]
88it [00:00, 116.13it/s]
848it [00:07, 115.07it/s]
212it [00:01, 122.13it/s]
3034it [00:24, 125.48it/s]
1260it [00:09, 128.20it/s]


In [50]:
Image(url="regions/version3/kirkland.png", width=400, height=300)

In [51]:
uppereast_trips_handler(read_regions('regions/version3/kirkland.txt'))

193it [00:01, 122.90it/s]
486it [00:03, 126.31it/s]
121it [00:00, 126.13it/s]
1039it [00:08, 126.84it/s]
259it [00:02, 127.34it/s]
3041it [00:25, 120.96it/s]
1576it [00:12, 127.18it/s]


In [52]:
Image(url="regions/version3/redmond.png", width=500, height=300)

In [53]:
uppereast_trips_handler(read_regions('regions/version3/redmond.txt'))

127it [00:01, 122.82it/s]
292it [00:02, 128.52it/s]
73it [00:00, 127.68it/s]
604it [00:04, 127.16it/s]
151it [00:01, 126.60it/s]
1802it [00:14, 128.51it/s]
904it [00:07, 127.61it/s]


In [54]:
Image(url="regions/version3/sammamish.png", width=500, height=300)

In [55]:
east_trips_handler(read_regions('regions/version3/sammamish.txt'))

265it [00:02, 124.39it/s]
7932it [01:02, 126.54it/s]


In [56]:
Image(url="regions/version3/bel1.png", width=400, height=300)

In [57]:
uppereast_trips_handler(read_regions('regions/version3/bel1.txt'))

735it [00:06, 118.68it/s]
936it [00:07, 126.48it/s]
233it [00:01, 125.76it/s]
2373it [00:18, 127.00it/s]
593it [00:04, 128.77it/s]
5574it [00:51, 108.90it/s]
2732it [00:27, 97.90it/s] 


In [58]:
Image(url="regions/version3/bel2.png", width=400, height=300)

In [59]:
east_trips_handler(read_regions('regions/version3/bel2.txt'))

604it [00:06, 95.21it/s] 
16321it [02:10, 124.78it/s]


In [60]:
Image(url="regions/version3/east5.png", width=400, height=300)

In [61]:
east_trips_handler(read_regions('regions/version3/east5.txt'))

399it [00:04, 97.23it/s] 
12115it [01:51, 108.31it/s]


In [62]:
Image(url="regions/version3/east41.png", width=400, height=300)

In [63]:
east_trips_handler(read_regions('regions/version3/east41.txt'))

154it [00:01, 95.28it/s]
4541it [00:36, 125.46it/s]


In [64]:
Image(url="regions/version3/east3.png", width=300, height=300)

In [65]:
southeast_trips_handler(read_regions('regions/version3/east3.txt'))

145it [00:01, 98.76it/s] 
178it [00:01, 96.88it/s]
3648it [00:28, 126.60it/s]
3649it [00:28, 127.31it/s]


In [66]:
Image(url="regions/version3/east2.png", width=400, height=300)

In [67]:
southeast_trips_handler(read_regions('regions/version3/east2.txt'))

99it [00:01, 95.32it/s]
96it [00:00, 96.53it/s]
3618it [00:29, 122.13it/s]
3619it [00:28, 128.57it/s]


In [68]:
Image(url="regions/version3/east1.png", width=400, height=300)

In [69]:
southeast_trips_handler(read_regions('regions/version3/east1.txt'))

38it [00:00, 108.39it/s]
42it [00:00, 127.72it/s]
874it [00:06, 130.79it/s]
874it [00:06, 127.33it/s]


### Old super segments
---------

In [70]:
Image(url="regions/north2.png", width=300, height=300)

In [71]:
north2 = read_regions('regions/north2.txt')+[358,359]
north2boundary = read_regions('regions/north2_boundary.txt')
# convert1(north2, north2boundary, 7002, 7002)

#v.3
north1 = [426, 427, 428, 429, 549, 551, 553]
north3=[436, 437, 438, 439, 440, 441, 442, 443]
north1boundary = read_regions('regions/north1_boundary.txt')
north3boundary = read_regions('regions/north3_boundary.txt')

df = converted_data[converted_data.otaz.isin(north2)|converted_data.dtaz.isin(north2)]


df1 = df[df.otaz.isin(north1)|df.dtaz.isin(north1)]
df1_1, df1_2 = random_split_dataframe(df1, 50)
convert1(north2, north2boundary, 7002, 7002, df1_1)
convert1(north2, north1boundary, 7001, 7001, df1_2)

df3 = df[df.otaz.isin(north3)|df.dtaz.isin(north3)]
df3_1, df3_2 = random_split_dataframe(df3, 50)
convert1(north2, north2boundary, 7002, 7002, df3_1)
convert1(north2, north3boundary, 7003, 7003, df3_2)


convert1(north2, north2boundary, 7002, 7002)

620it [00:04, 125.31it/s]
621it [00:04, 126.30it/s]
571it [00:04, 126.00it/s]
572it [00:04, 124.96it/s]
15587it [02:12, 117.23it/s]


In [72]:
Image(url="regions/SR99N.png", width=300, height=300)

In [73]:
SR99n = read_regions('regions/SR99N.txt')
nearSR99n = read_regions('regions/near_SR99N.txt')

In [74]:
convert1(SR99n, nearSR99n, 5010, 5011)

0it [00:00, ?it/s]


 --------

In [75]:
Image(url="regions/i5n1.png", width=300, height=300)

In [76]:
i5n1 = read_regions('regions/i5n1.txt')

In [77]:
convert1(i5n1, near_i5n, 5000, 5001)

0it [00:00, ?it/s]


___

In [78]:
Image(url="regions/i5n2.png", width=300, height=300)

In [79]:
i5n2 = read_regions('regions/i5n2.txt')
convert1(i5n2, near_i5n, 5000, 5001)

0it [00:00, ?it/s]


--------

In [80]:
Image(url="regions/west_king1.png", width=600, height=300)

In [81]:
west_king1 = read_regions('regions/west_king1.txt')
df = converted_data[converted_data.otaz.isin(west_king1)|converted_data.dtaz.isin(west_king1)]

In [82]:
# if trips go to upper downtown:
df1 = df[df.otaz.isin(sumo_n)|df.dtaz.isin(sumo_n)]
convert1(west_king1, near_i5n, 5000, 5001, df=df1)

51it [00:00, 96.53it/s] 


In [83]:
# if trips go to lower downtown:
df2 = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
convert1(west_king1, near_i90, 5004, 5005, df=df2)

25it [00:00, 93.59it/s]


---------

In [84]:
Image(url="regions/west_king2.png", width=600, height=300)

In [85]:
west_king2 = read_regions('regions/west_king2.txt')
df = converted_data[converted_data.otaz.isin(west_king2)|converted_data.dtaz.isin(west_king2)]

In [86]:
# if trips go to upper downtown:
df1 = df[df.otaz.isin(sumo_n)|df.dtaz.isin(sumo_n)]
convert1(west_king2, near_i5n, 5000, 5001, df=df1)

0it [00:00, ?it/s]


In [87]:
# if trips go to lower downtown:
df2 = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
convert1(west_king2, near_i90, 5004, 5005, df=df2)

0it [00:00, ?it/s]


-----

In [88]:
Image(url="regions/west_king3.png", width=500, height=300)

In [89]:
west_king3 = read_regions('regions/west_king3.txt')
convert1(west_king3, near_i90, 5004, 5005)

93it [00:00, 97.22it/s]


-----

In [90]:
Image(url="regions/west_king4.png", width=600, height=300)

In [91]:
west_king4 = read_regions('regions/west_king4.txt')
convert1(west_king4, near_i90, 5004, 5005)

0it [00:00, ?it/s]


-----

In [92]:
Image(url="regions/west_king5.png", width=600, height=300)

In [93]:
west_king5 = read_regions('regions/west_king5.txt')
convert1(west_king5, near_i5s, 5002, 5003)

4920it [00:46, 106.22it/s]


-----

In [94]:
Image(url="regions/west_king6.png", width=300, height=300)

In [95]:
west_king6 = read_regions('regions/west_king6.txt')
convert1(west_king6, near_i5s, 5002, 5003)

863it [00:08, 104.94it/s]


--------

In [96]:
Image(url="regions/vashon.png", width=300, height=300)

In [97]:
vashon = read_regions('regions/vashon.txt')
nearsr99s = read_regions('regions/near_SR99S.txt')
convert1(vashon, nearsr99s, 5008, 5009)

1206it [00:10, 115.51it/s]


------

In [98]:
Image(url="regions/southwest1.png", width=300, height=300)

In [99]:
sw1 = read_regions('regions/southwest1.txt')
df = converted_data[converted_data.otaz.isin(sw1)|converted_data.dtaz.isin(sw1)]

In [100]:
# if trips go to west downtown:
df1 = df[df.otaz.isin(sumo_w)|df.dtaz.isin(sumo_w)]
# convert1(sw1, nearsr99s, 5008, 5009, df=df1)

# v.3
df12 = df1[df1.otaz.isin(industrial)|df1.dtaz.isin(industrial)]
convert1(sw1, nearsr99s, [5008, 7000], [5009, 7000], df=df12)

convert1(sw1, nearsr99s, 5008, 5009, df=df1)

3303it [00:25, 127.83it/s]
19529it [02:27, 132.24it/s]


In [101]:
# if trips go to east downtown:
df2 = df[df.otaz.isin(sumo_e)|df.dtaz.isin(sumo_e)]
convert1(sw1, near_i5s, 5002, 5003, df=df2)

4512it [00:35, 126.36it/s]


In [102]:
Image(url="regions/southwest2.png", width=300, height=300)


In [103]:
sw2 = read_regions('regions/southwest2.txt')
df = converted_data[converted_data.otaz.isin(sw2)|converted_data.dtaz.isin(sw2)]

In [104]:
# if trips go to west downtown:
df1 = df[df.otaz.isin(sumo_w)|df.dtaz.isin(sumo_w)]
# convert1(sw2, nearsr99s, 5008, 5009, df=df1)

# v.3
df12 = df1[df1.otaz.isin(industrial)|df1.dtaz.isin(industrial)]
convert1(sw2, nearsr99s, [5008, 7000], [5009, 7000], df=df12)

convert1(sw2, nearsr99s, 5008, 5009, df=df1)

5939it [00:46, 128.17it/s]
30346it [03:51, 131.36it/s]


In [105]:
# if trips go to east downtown:
df2 = df[df.otaz.isin(sumo_e)|df.dtaz.isin(sumo_e)]
convert1(sw2, near_i5s, 5002, 5003, df=df2)

6100it [00:47, 128.75it/s]


----

In [106]:
Image(url="regions/version3/east42.png", width=400, height=300)

In [107]:
reg = read_regions('regions/version3/east42.txt')

# v.3
df = converted_data[converted_data.otaz.isin(reg)|converted_data.dtaz.isin(reg)]
df1 = df[df.otaz.isin(sumo_e)|df.dtaz.isin(sumo_e)]
convert1(reg, near_i5s, 5002, 5003, df=df1)

df1 = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
convert1(reg, near_i5s, [5002, 7000], [5003, 7000], df=df1)

df = converted_data[converted_data.otaz.isin(reg)|converted_data.dtaz.isin(reg)]
df20, df80 = random_split_dataframe(df, 20)
convert1(reg, near_i5s, 5002, 5003, df=df80)
convert1(reg, nearsr99s, 5008, 5009, df=df20)

1521it [00:12, 122.49it/s]
2700it [00:21, 126.92it/s]
2671it [00:20, 127.70it/s]
667it [00:05, 127.87it/s]


In [108]:
Image(url="regions/i5s11.png", width=300, height=1000)

In [109]:
i5s1 = read_regions('regions/i5s1.txt')

reg = read_regions('regions/i5s1.txt')
# convert1(i5s1, near_i5s, 5002, 5003)

# v.3
df = converted_data[converted_data.otaz.isin(reg)|converted_data.dtaz.isin(reg)]
df1 = df[df.otaz.isin(sumo_e)|df.dtaz.isin(sumo_e)]
convert1(reg, near_i5s, 5002, 5003, df=df1)

# df1 = df[df.otaz.isin(industrial)|df.dtaz.isin(industrial)]
df1 = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
convert1(reg, near_i5s, [5002, 7000], [5003, 7000], df=df1)

df = converted_data[converted_data.otaz.isin(reg)|converted_data.dtaz.isin(reg)]
df20, df80 = random_split_dataframe(df, 20)
convert1(reg, near_i5s, 5002, 5003, df=df80)
convert1(reg, nearsr99s, 5008, 5009, df=df20)


4229it [00:34, 122.41it/s]
7743it [01:04, 120.70it/s]
7631it [01:19, 96.03it/s] 
1907it [00:19, 96.92it/s] 


In [110]:
Image(url="regions/i5s2.png", width=300, height=300)

In [111]:
i5s2 = read_regions('regions/i5s2.txt')
# convert1(i5s2, near_i5s, 5002, 5003)

# v.3
df = converted_data[converted_data.otaz.isin(i5s2)|converted_data.dtaz.isin(i5s2)]
# df = df[df.otaz.isin(sumo_w)|df.dtaz.isin(sumo_w)]
# df = df[df.otaz.isin(industrial+sumo_n)|df.dtaz.isin(industrial+sumo_n)]
df = df[df.otaz.isin(industrial)|df.dtaz.isin(industrial)]
df30, df70 = random_split_dataframe(df, 30)
convert1(i5s2, near_i5s, 7000, 7000, df=df30)


convert1(i5s2, near_i5s, 5002, 5003)


462it [00:04, 97.22it/s] 
13769it [01:50, 125.05it/s]


-----

In [112]:
Image(url="regions/southeast.png", width=400, height=300)

In [113]:
se = read_regions('regions/southeast.txt')

In [114]:
# if trips go to east downtown:
df = converted_data[converted_data.otaz.isin(se)|converted_data.dtaz.isin(se)]
df = df[df.otaz.isin(sumo_e)|df.dtaz.isin(sumo_e)]
# convert1(se, [636], 636, 636, df=df)

#v.3
east2boundary = read_regions('regions/east2_boundary.txt')
convert1(se, east2boundary, 7005, 7005, df=df)

4056it [00:32, 123.43it/s]


In [115]:
# if trips go to south downtown:
df = converted_data[converted_data.otaz.isin(se)|converted_data.dtaz.isin(se)]
df = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
# convert1(se, [650], 650, 650, df=df)

#v.3
southboundary = read_regions('regions/south_boundary.txt')
df1, df2 = random_split_dataframe(df, 50)
convert1(se, east2boundary, 7005, 7005, df1)
convert1(se, southboundary, 7000,  7000, df2)

2535it [00:20, 124.66it/s]
2535it [00:20, 124.58it/s]


In [116]:
# if trips go to north downtown:
df = converted_data[converted_data.otaz.isin(se)|converted_data.dtaz.isin(se)]
df = df[df.otaz.isin(sumo_n)|df.dtaz.isin(sumo_n)]
convert1(se, near_i5s, 5002, 5003, df=df)

6196it [00:46, 132.89it/s]


----

In [117]:
Image(url="regions/east1.png", width=400, height=300)

In [118]:
east1 = read_regions('regions/east1.txt')
boundary = read_regions('regions/east1_boundary.txt')
# convert1(east1, boundary, 7004, 7004)

#v.3 - for trips to sumo south
df = converted_data[converted_data.otaz.isin(east1)|converted_data.dtaz.isin(east1)]
df = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
df30, df70 = random_split_dataframe(df, 30)
convert1(east1, boundary, 7005, 7005, df=df70)

convert1(east1, boundary, 7004, 7004)

6070it [00:48, 126.41it/s]
27510it [03:44, 122.38it/s]


----

In [119]:
Image(url="regions/east2.png", width=400, height=300)

In [120]:
east2 = read_regions('regions/east2.txt')
boundary = read_regions('regions/east2_boundary.txt')
# convert1(east2, boundary, 7005, 7005)

#v.3 - for trips to sumo northeast
df = converted_data[converted_data.otaz.isin(east2)|converted_data.dtaz.isin(east2)]
df = df[df.otaz.isin(sumo_n)|df.dtaz.isin(sumo_n)]
df = df[df.otaz.isin(sumo_e)|df.dtaz.isin(sumo_e)]
df1, df2 = random_split_dataframe(df, 50)
convert1(east2, boundary, 7004, 7004, df=df1)  

convert1(east2, boundary, 7005, 7005)

1447it [00:13, 111.23it/s]
16623it [02:06, 131.64it/s]


----

In [121]:
Image(url="regions/north1.png", width=400, height=300)

In [122]:
north1 = read_regions('regions/north1.txt')
boundary = read_regions('regions/north1_boundary.txt')
# convert1(north1, boundary, boundary, boundary)
convert1(north1, boundary, 7001, 7001)

8357it [01:04, 129.95it/s]


----

----

In [123]:
Image(url="regions/north_west.png", width=300, height=300)

In [124]:
NW = read_regions('regions/north_west.txt')
boundary = read_regions('regions/north3_boundary.txt')
df = converted_data[converted_data.otaz.isin(NW)|converted_data.dtaz.isin(NW)]

df1 = df[df.otaz.isin(sumo_n)|df.dtaz.isin(sumo_n)]
# convert1(NW, boundary, boundary, boundary, df=df1)
convert1(NW, boundary, 7003, 7003, df=df1)

df2 = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
convert1(NW, nearSR99n, 5010, 5011, df=df2)

21580it [02:41, 133.70it/s]
3582it [00:25, 138.08it/s]


In [125]:
convert1(NW, nearSR99n, 5010, 5011)

0it [00:00, ?it/s]


In [126]:
converted_data[converted_data.otaz.isin(NW)|converted_data.dtaz.isin(NW)]


,otaz,dtaz,deptm,travtime,original_o,original_d,original_deptm


In [127]:
converted_data[converted_data.otaz.isin(NW)|converted_data.dtaz.isin(NW)]


,otaz,dtaz,deptm,travtime,original_o,original_d,original_deptm


-------

In [128]:
Image(url="regions/6001.png", width=300, height=300)

In [129]:
to6001 = read_regions('regions/6001.txt')
near6001 = read_regions('regions/near6001.txt')

In [130]:
# df = converted_data[converted_data.otaz.isin(to6001)|converted_data.dtaz.isin(to6001)]

# df1 = df[df.otaz.isin(sumo_n)|df.dtaz.isin(sumo_n)]
# convert1(to6001, near6001, 6001, 6001, df=df1)

# df2 = df[df.otaz.isin(sumo_s)|df.dtaz.isin(sumo_s)]
# convert1(to6001, nearSR99n, 5010, 5011, df=df2)

df = converted_data[converted_data.otaz.isin(to6001)|converted_data.dtaz.isin(to6001)]
df1 = df[df.otaz.isin(industrial)|df.dtaz.isin(industrial)]
convert1(to6001, nearSR99n, 5010, 5011, df=df1)

north3=[436, 437, 438, 439, 440, 441, 442, 443]
df = converted_data[converted_data.otaz.isin(to6001)|converted_data.dtaz.isin(to6001)]
df = df[df.otaz.isin(north3)|df.dtaz.isin(north3)]
convert1(to6001, near6001, [6001, 7003], [6001, 7003], df=df)

convert1(to6001, near6001, 6001, 6001)

904it [00:06, 137.79it/s]
2074it [00:14, 138.39it/s]
12884it [01:33, 138.15it/s]


----

In [131]:
Image(url="regions/south.png", width=300, height=300)

In [132]:
south = read_regions('regions/south.txt')
boundary = read_regions('regions/south_boundary.txt')
# convert1(south, boundary, boundary, boundary)
convert1(south, boundary, 7000, 7000)

11355it [01:21, 138.55it/s]


-------

## TAZ conversion for trips passing through sumo_TAZs

For this stage, we assume some scenarios, find related trips in out main trip dataframe, process them, and then add them to our existing trips

In [133]:
def get_trips_in_between(src_df, taz_list1, taz_list2):
# make sure taz_list1 and taz_list2 have no TAZ within sumo_TAZs. 
# In that case, some of the trips that were already processed in converted_data will be included
    df = src_df[(src_df.otaz.isin(taz_list1) & src_df.dtaz.isin(taz_list2)) | 
                (src_df.otaz.isin(taz_list2) & src_df.dtaz.isin(taz_list1))]
    return df

In [134]:
def convert2(df, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2):
    zone1offsets = get_intra_TAZ_traveltime(zone1, near_new_z1)
    zone2offsets = get_intra_TAZ_traveltime(zone2, near_new_z2)
    new_z1_in, new_z1_out = new_z1[0], new_z1[1]
    new_z2_in, new_z2_out = new_z2[0], new_z2[1]
    
    print(f"it'll take {len(df)} iterations.")
    for index, row in tqdm(df.iterrows()):
        interval = int(row.deptm//15)
        
        if row.otaz in zone1 and row.dtaz in zone2:
            row.otaz = get_random_TAZ(new_z1_in)
            row.dtaz = get_random_TAZ(new_z2_out)
            row.deptm += zone1offsets[interval]
            
        elif row.otaz in zone2 and row.dtaz in zone1:
            row.otaz = get_random_TAZ(new_z2_in)
            row.dtaz = get_random_TAZ(new_z1_out)
            row.deptm += zone2offsets[interval]
#         converted_data.loc[index, :] = row 
        df.loc[index, :] = row

-----

In [135]:
Image(url="regions/scenarios.png", width=900, height=300)

In [136]:
#scenario 1:

# out_of_sumo_data = get_trips_in_between(trip_data, zone1, zone2) 
out_of_sumo_data = get_trips_in_between(trip_data, i5n2, i5s1) 
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
convert2(out_of_sumo_data, i5n2, [5000, 5001], near_i5n, i5s1, [5002, 5003], near_i5s)

it'll take 1406 iterations.


1406it [00:00, 1498.10it/s]


In [137]:
def make_pseudo_trips(zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2):
        df = get_trips_in_between(trip_data, zone1, zone2) 
        convert2(df, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
        return df

In [138]:
# scenario 2
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
df = make_pseudo_trips(i5n2, [5000, 5001], near_i5n, sw2, [5002, 5003], near_i5s)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])


it'll take 2462 iterations.


2462it [00:01, 1538.65it/s]


In [139]:
# scenario 3
east2_b = read_regions('regions/east2_boundary.txt')

# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(i5n2, [5000, 5001], near_i5n, east2, [east2_b, east2_b], east2_b)
df = make_pseudo_trips(i5n2, [5000, 5001], near_i5n, east2, [7005, 7005], east2_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 1301 iterations.


1301it [00:00, 1584.28it/s]


In [140]:
# scenario 4
east1_b = read_regions('regions/east1_boundary.txt')
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(i5n2, [5000, 5001], near_i5n, east1, [east1_b, east1_b], east1_b)
df = make_pseudo_trips(i5n2, [5000, 5001], near_i5n, east1, [7004, 7004], east1_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 2657 iterations.


2657it [00:01, 1513.06it/s]


In [141]:
# scenario 5
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(i5s2, [5002, 5003], near_i5s, east1, [east1_b, east1_b], east1_b)
df = make_pseudo_trips(i5s2, [5002, 5003], near_i5s, east1, [7004, 7004], east1_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 2196 iterations.


2196it [00:01, 1503.24it/s]


In [142]:
# scenario 6
north3_b = read_regions('regions/north3_boundary.txt')
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(NW, [north3_b, north3_b], north3_b, east1, [east1_b, east1_b], east1_b)
df = make_pseudo_trips(NW, [7003, 7003], north3_b, east1, [7004, 7004], east1_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 1588 iterations.


1588it [00:01, 1516.64it/s]


In [143]:
# scenario 7
north2_b = read_regions('regions/north2_boundary.txt')
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(north2, [north2_b, north2_b], north2_b, east2, [east2_b, east2_b], east2_b)
df = make_pseudo_trips(north2, [7002, 7002], north2_b, east2, [7005, 7005], east2_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 944 iterations.


944it [00:00, 1570.13it/s]


In [144]:
# scenario 8
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(NW, [north3_b, north3_b], north3_b, east2, [east2_b, east2_b], east2_b)
df = make_pseudo_trips(NW, [7003, 7003], north3_b, east2, [7005, 7005], east2_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 851 iterations.


851it [00:00, 1486.74it/s]


In [145]:
# scenario 9
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(i5s2, [5002, 5003], near_i5s, east2, [east2_b, east2_b], east2_b)
df = make_pseudo_trips(i5s2, [5002, 5003], near_i5s, east2, [7005, 7005], east2_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 1846 iterations.


1846it [00:01, 1518.82it/s]


In [146]:
# scenario 10
south_b = read_regions('regions/south_boundary.txt')
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(NW, [north3_b, north3_b], north3_b, south, [south_b, south_b], south_b)
df = make_pseudo_trips(NW, [7003, 7003], north3_b, south, [7000, 7000], south_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 1172 iterations.


1172it [00:00, 1542.44it/s]


In [147]:
# scenario 11
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
df = make_pseudo_trips(NW, [436, 436], north3_b, se, [[636, 650], [636, 650]], [636, 647, 648, 649, 650 ])
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 624 iterations.


624it [00:00, 1497.96it/s]


In [148]:
# scenario 12
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(NW, [north3_b, north3_b], north3_b, sw1, [5008, 5009], nearsr99s)
df = make_pseudo_trips(NW, [7003, 7003], north3_b, sw1, [5008, 5009], nearsr99s)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 2238 iterations.


2238it [00:01, 1516.67it/s]


In [149]:
# scenario 13
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
df = make_pseudo_trips((SR99n+to6001), [5010, 5011], nearSR99n,
                       sw1, [5008, 5009], nearsr99s)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 9355 iterations.


9355it [00:06, 1384.99it/s]


In [150]:
# scenario 14
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
df = make_pseudo_trips(i5n1, [5000, 5001], near_i5n,
                       i5s1, [5002, 5003], near_i5s)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 7321 iterations.


7321it [00:05, 1424.86it/s]


In [151]:
# scenario 15
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(i5n1, [5000, 5001], near_i5n, south, [south_b, south_b], south_b)
df = make_pseudo_trips(i5n1, [5000, 5001], near_i5n, 
                       south, [7000, 7000], south_b)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 4856 iterations.


4856it [00:03, 1444.53it/s]


In [152]:
# scenario 18
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
df = make_pseudo_trips(sw2, [5008, 5009], nearsr99s,
                       (SR99n+to6001), [5010, 5011], nearSR99n)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 14977 iterations.


14977it [00:11, 1278.01it/s]


In [153]:
# scenario 19
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(south, [south_b, south_b], south_b,
#                        (SR99n+to6001), [5010, 5011], nearSR99n)
df = make_pseudo_trips(south, [7000, 7000], south_b,
                       (SR99n), [5010, 5011], nearSR99n)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 4004 iterations.


4004it [00:02, 1503.19it/s]


In [154]:
# scenario 21
df = make_pseudo_trips(south, [7000, 7000], south_b, to6001, [6001, 6001], near6001)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 3002 iterations.


3002it [00:01, 1526.41it/s]


In [155]:
# scenario 20
north1_b = read_regions('regions/north1_boundary.txt')
# convert2(out_of_sumo_data, zone1, new_z1, near_new_z1, zone2, new_z2, near_new_z2)
# df = make_pseudo_trips(south, [south_b, south_b], south_b,
#                        north1, [5000, 5001], near_i5n)
df = make_pseudo_trips(south, [7000, 7000], south_b,
                       north1, [5000, 5001], near_i5n)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 388 iterations.


388it [00:00, 1444.47it/s]


In [156]:
# scenario 22
df = make_pseudo_trips(south, [7000, 7000], south_b, north2, [7002, 7002], north2boundary)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 689 iterations.


689it [00:00, 1498.82it/s]


In [157]:
# scenario 23
df = make_pseudo_trips(to6001, [6001, 6001], near6001,
                       east2, [7005, 7005], east2boundary)
out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 1725 iterations.


1725it [00:01, 1503.98it/s]


In [158]:
# scenario 24  -- south bellevue to south
bel2 = read_regions('regions/version3/bel2.txt')
bel3 = read_regions('regions/version3/east5.txt')
df = make_pseudo_trips(south, [7000, 7000], south_b, 
                       (bel2+bel3), [5004, 5005], near_i90)

out_of_sumo_data = pd.concat([out_of_sumo_data, df])

it'll take 5502 iterations.


5502it [00:03, 1471.48it/s]


In [159]:
out_of_sumo_data

,otaz,dtaz,deptm,travtime,original_o,original_d,original_deptm
24739,5000,5003,490,28,8,708,464
24740,5000,5003,506,30,8,709,478
24741,5000,5003,529,30,8,709,503
24742,5000,5003,887,24,8,711,869
24745,5000,5003,1001,36,8,718,977
...,...,...,...,...,...,...,...
3918879,5004,7000,365,24,1540,670,340
3918880,5004,7000,670,22,1540,671,653
3918881,5004,7000,1285,21,1540,671,1272
3918885,5004,7000,534,26,1540,705,515


In [160]:
len(converted_data)

608908

In [161]:
df = pd.concat([converted_data, out_of_sumo_data])

In [162]:
len(df)

680012

In [176]:
df[df.otaz == 402]

,otaz,dtaz,deptm,travtime,original_o,original_d,original_deptm


# making CSV file

In [163]:
df.to_csv('psrc_vehicle_trips_converted_taz_calibrated_v2.csv', index=False)